In [76]:
%load_ext autoreload
%autoreload 2

%pip install -r requirements.txt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Note: you may need to restart the kernel to use updated packages.


In [77]:
import sys
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision
import torchvision.models as models
from torchvision import transforms
from datasets import load_dataset, concatenate_datasets
import pickle

In [78]:
# load in the pretrained vgg code
import shutil

if not os.path.exists('cifar10_models'):
    !git clone https://github.com/huyvnphan/PyTorch_CIFAR10
    
    # copy cifar10_models folder to current directory
    shutil.copytree(os.path.join("PyTorch_CIFAR10", "cifar10_models"), "cifar10_models")
    
    # delete the cloned repo
    shutil.rmtree("PyTorch_CIFAR10")
    

from cifar10_models import *

In [79]:
print(f"PyTorch version: {torch.__version__}")

# Check PyTorch has access to MPS (Metal Performance Shader, Apple's GPU architecture)
print(f"Is MPS (Metal Performance Shader) built? {torch.backends.mps.is_built()}")
print(f"Is MPS available? {torch.backends.mps.is_available()}")

# Check for CUDA support
print(f"Is CUDA available? {torch.cuda.is_available()}")

# Set the device
if torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

print(f"Using device: {device}")


PyTorch version: 2.1.0
Is MPS (Metal Performance Shader) built? True
Is MPS available? True
Is CUDA available? False
Using device: mps


In [80]:
from EarlyExitModel import EarlyExitModel

def create_model_with_exits(model_type, num_classes):
    if model_type == "resnet":
        resnet = models.resnet50(pretrained=True)
        
        # set requires_grad to False to freeze the parameters
        for param in resnet.parameters():
            param.requires_grad = False
        
        resnet.fc = nn.Linear(resnet.fc.in_features, num_classes)
        
        model = EarlyExitModel(resnet, num_classes, device)
        model.clear_exits()
        exit_layers = [model.add_exit(layer, model_type) for layer in ('layer1', 'layer2', 'layer3')]
        
    elif model_type == "vgg_cifar10":
        # model loaded from https://github.com/huyvnphan/PyTorch_CIFAR10
        # use this code to generate a pretrained model and pickle/save it
        
        model_path = os.path.join('models', 'vgg_cifar10', 'vgg11_bn.pkl')
        vgg = pickle.load(open(model_path, 'rb'))
        # set requires_grad to False to freeze the parameters
        for param in vgg.parameters():
            param.requires_grad = False
        vgg.classifier[-1] = nn.Linear(vgg.classifier[-1].in_features, num_classes)
        
        model = EarlyExitModel(vgg, num_classes, device)
        model.clear_exits()
        exit_layers = [model.add_exit(layer, model_type) for layer in ('features.8', 'features.15', 'features.22', 'avgpool')]
    elif model_type == "vgg_cifar100":
        vgg = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar100_vgg11_bn", pretrained=True)
        # set requires_grad to False to freeze the parameters
        for param in vgg.parameters():
            param.requires_grad = False
        vgg.classifier[-1] = nn.Linear(vgg.classifier[-1].in_features, num_classes)
        model = EarlyExitModel(vgg, num_classes, device)
        model.clear_exits()
        exit_layers = [model.add_exit(layer, model_type) for layer in ('features.8', 'features.15', 'features.22', "classifier.0")]
    else:
        raise ValueError(f"Unknown model type: {model_type}")
    
    model.to(device)
    return model
        
        

In [81]:
# choose the model
model_type = "vgg_cifar100" # either "resnet" or "vgg_cifar10" or "vgg_cifar100"

model_path = os.path.join("models", model_type)

num_classes = 100 if model_type == "vgg_cifar100" else 10

model = create_model_with_exits(model_type, num_classes=num_classes)

model

Using cache found in /Users/dylanmace/.cache/torch/hub/chenyaofo_pytorch-cifar-models_master


EarlyExitModel(
  (model): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): ReLU(inplace=True)
      (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (8): OptionalExitModule(
        (module): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
      (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (10): ReLU(inplace=True)
      (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (12): BatchNorm2d(256, eps=1e-05, mom

In [82]:
from DataLoader import CustomDataset
import numpy as np

if model_type == "resnet":
    # use the imagenette dataset
    hf_dataset = load_dataset("frgfm/imagenette", '320px')
    hf_dataset = concatenate_datasets(hf_dataset.values())
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])

elif model_type == "vgg_cifar10":
    # use the cifar10 dataset
    hf_dataset = load_dataset("cifar10")
    hf_dataset = concatenate_datasets(hf_dataset.values())
    
    transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.507, 0.4865, 0.4409],
                             std=[0.2673, 0.2564, 0.2761])
    ])
elif model_type == "vgg_cifar100":
    # use the cifar100 dataset
    hf_dataset = load_dataset("cifar100")
    hf_dataset = concatenate_datasets(hf_dataset.values())
    
    transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
    ])
    
torch_dataset = CustomDataset(hf_dataset, transform=transform)

batch_size = 32 if model_type == "resnet" else 64

test_size = 0.2
test_volume = int(test_size * len(torch_dataset))
train_volume = len(torch_dataset) - test_volume

train_dataset, test_dataset = random_split(torch_dataset, [train_volume, test_volume])
train_dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=False, 
    num_workers=4
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=4
)

In [83]:
# # TODO: remove this cell
# import OptionalExitModule

# model = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar100_vgg11_bn", pretrained=True)
# model.to(device)
          
# losses = []
# accuracies = []
# loss_fn = nn.CrossEntropyLoss()
# for (X, y) in test_dataloader:
#     X = X.to(device)
#     y = y.to(device)
#     y_hat = model(X)
    
#     losses.append(loss_fn(y_hat, y).item())
#     accuracies.append((y_hat.argmax(dim=1) == y).float().mean().item())

# print(f"Loss: {np.mean(losses)}")
# print(f"Accuracy: {np.mean(accuracies)}")

## Early Exit Model Training

If you want tensorboard support, you need to run the following commands on **macOS**:

This clears your logs:
```
rm -rf runs/*
```

This shows your public IP address:
```
dig -4 TXT +short o-o.myaddr.l.google.com @ns1.google.com
```

This starts tensorboard at this address:
```
tensorboard --host 0.0.0.0 --logdir={model_path}/runs
```
```



If you want tensorboard support, you need to run the following commands on **Windows**:

This clears your logs:
```
rmdir /s /q runs
```

This shows your IP address:
```
powershell -command "$ipAddress = (Invoke-WebRequest -Uri 'http://ipinfo.io/ip').Content.Trim(); Write-Host 'Your IP address is: ' $ipAddress"  
```

This starts tensorboard at this address:
```
tensorboard --host 0.0.0.0 --logdir={model_path}\runs
```

In [84]:
from EarlyExitTrainer import ModelTrainer

trainer = ModelTrainer(model, device, model_dir=model_path)

In [85]:
# train the classifiers
trainer.train_classifiers(train_dataloader, epoch_count=30, validation_loader=test_dataloader)


Training classifier for exit 1


Epoch 0:   0%|                                                              | 0/750 [00:00<?, ?it/s]

Epoch 0 Loss 3.9712005904515584
Epoch 0 Accuracy 0.16116666666666668


Epoch 0 Validation Loss 3.44422343817163
Epoch 0 Validation Accuracy 0.2598902925531915


Epoch 1 Loss 3.197557509740194
Epoch 1 Accuracy 0.28639583333333335


Epoch 1 Validation Loss 3.037212960263516
Epoch 1 Validation Accuracy 0.3074301861702128


Epoch 2 Loss 2.9093573627471923
Epoch 2 Accuracy 0.32958333333333334


In [ ]:
# train the exits

#trainer.train_exit_layers(train_dataloader, epoch_count=30, validation_loader=test_dataloader)